In [1]:
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn
# %pip install kagglehub
# %pip install cupy

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import kagglehub
import cupy as cp


In [3]:
historical_hourly_weather_data_path = kagglehub.dataset_download('selfishgene/historical-hourly-weather-data')

city = "Portland"

city_attributes = pd.read_csv(f"{historical_hourly_weather_data_path}/city_attributes.csv")
humidity = pd.read_csv(f"{historical_hourly_weather_data_path}/humidity.csv")
pressure = pd.read_csv(f"{historical_hourly_weather_data_path}/pressure.csv")
temperature = pd.read_csv(f"{historical_hourly_weather_data_path}/temperature.csv")
weather_description = pd.read_csv(f"{historical_hourly_weather_data_path}/weather_description.csv")
wind_speed = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_speed.csv")
wind_direction = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_direction.csv")

In [4]:
if city not in city_attributes['City'].values:
    raise ValueError(f"City '{city}' does not exist in the data. Available cities are: {city_attributes['City'].unique()}")

selected_city = city_attributes[city_attributes['City'] == city].index[0]
data_frames = [humidity, pressure, temperature, weather_description, wind_speed, wind_direction]

for i, df in enumerate(data_frames):
    df.set_index('datetime', inplace=True)
    data_frames[i] = df.iloc[:, selected_city]

In [5]:
combined_data = pd.concat(data_frames, axis=1)
combined_data.columns = [
    'humidity', 'pressure', 'temperature', 'weather_description', 
    'wind_speed', 'wind_direction'
]
combined_data.index = pd.to_datetime(combined_data.index)
# aggregate daily
aggregated_data = combined_data.resample('D').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'wind_speed': ['max', 'mean'],
    'pressure': 'mean',
    'weather_description': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'wind_direction': 'mean'
})
aggregated_data.columns = ['_'.join(col).strip('_') for col in aggregated_data.columns.values]
aggregated_data = aggregated_data.rename(columns={
    'temperature_mean': 'mean_temperature',
    'wind_speed_max': 'max_wind_speed'
})

In [6]:
# def get_wind_direction_for_max_speed(group):
#     max_wind_speed_idx = group['wind_speed'].idxmax()
#     return group.loc[max_wind_speed_idx, 'wind_direction']

In [7]:
# aggregated_data.rename(columns={'wind_speed': 'wind_speed_max'}, inplace=True)

# aggregated_data['wind_direction'] = combined_data.groupby(combined_data.index.date).apply(get_wind_direction_for_max_speed)

# aggregated_data['wind_speed_mean'] = combined_data['wind_speed'].resample('D').mean()

# encoder = LabelEncoder()
# aggregated_data['weather_description'] = encoder.fit_transform(aggregated_data['weather_description'])

In [8]:
# weather_mapping = dict(enumerate(encoder.classes_))
# print("Mapping for weather_description:", weather_mapping)

In [9]:
# aggregated_data['mean_temperature_next_day'] = aggregated_data['temperature'].shift(-1)
# aggregated_data['max_wind_speed_next_day'] = aggregated_data['wind_speed_max'].shift(-1)

# aggregated_data = aggregated_data.dropna()

In [10]:
# X = aggregated_data.drop(columns=['mean_temperature_next_day', 'max_wind_speed_next_day'])
# y = aggregated_data[['mean_temperature_next_day', 'max_wind_speed_next_day']]

# train_size = int(0.8 * len(aggregated_data))
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

In [11]:
# print("Train data:", X_train.shape, y_train.shape)
# print("Test data:", X_test.shape, y_test.shape)

In [12]:
# print(X_train)

In [13]:
# print(y_train)

In [14]:
# def create_time_windows(data, window_size=5):
#     X, y = [], []
#     for i in range(window_size, len(data)):
#         X.append(data.iloc[i-window_size:i][['temperature', 'humidity', 'pressure', 'wind_speed_max', 'wind_speed_mean', 'wind_direction', 'weather_description']].values.T)
#         y.append(data.iloc[i][['mean_temperature_next_day', 'max_wind_speed_next_day']].values)
#     return np.array(X), np.array(y)

In [15]:
# X, y = create_time_windows(aggregated_data)

# train_size = int(0.8 * len(X))
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

In [16]:
# print("Training data windowed:", X_train.shape, y_train.shape)
# print("Test data windowed:", X_test.shape, y_test.shape)

In [17]:
def preprocess_weather_data(data, window_size=3):
    X, y = [], []
    for i in range(window_size, len(data) - 1):
        X_window = data.iloc[i-window_size:i][[
            'mean_temperature', 'humidity_mean', 'pressure_mean', 'max_wind_speed', 'wind_speed_mean', 'wind_direction_mean'
        ]].values
        y_target = data.iloc[i][['mean_temperature', 'max_wind_speed']].values

        # encode wind speed > 6 as binary
        y_target[1] = 1 if y_target[1] >= 6 else 0
        X.append(X_window)
        y.append(y_target)

    X = np.array(X)
    y = np.array(y)

    # X_mean = X.mean(axis=(0, 1), keepdims=True)
    # X_std = X.std(axis=(0, 1), keepdims=True)
    # X_normalized = (X - X_mean) / (X_std + 1e-9)

    train_size = int(0.7 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    X_mean = X_train.mean(axis=(0, 1), keepdims=True)
    X_std = X_train.std(axis=(0, 1), keepdims=True)
    X_train = (X_train - X_mean) / (X_std + 1e-9)
    X_test = (X_test - X_mean) / (X_std + 1e-9)

    return X_train, X_test, y_train, y_test

In [18]:
# X, y = preprocess_weather_data(aggregated_data, window_size=3)

# # Split into train/test
# train_size = int(0.8 * len(X))
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

In [19]:
X_train, X_test, y_train, y_test = preprocess_weather_data(aggregated_data, window_size=3)

In [20]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(1318, 3, 6)
(1318, 2)
(565, 3, 6)
(565, 2)


In [21]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=1000, learning_rate=0.001,rate = 500):
    X_train_cp = cp.array(X_train.reshape(X_train.shape[0], -1), dtype=cp.float32)
    y_train_cp = cp.array(y_train, dtype=cp.float32)
    X_test_cp = cp.array(X_test.reshape(X_test.shape[0], -1), dtype=cp.float32)
    y_test_cp = cp.array(y_test, dtype=cp.float32)

    model.train(X_train_cp, y_train_cp, epochs, learning_rate,rate)

    predictions = model.predict(X_test_cp)
    
    mae = cp.mean(cp.abs(predictions[:, 0] - y_test_cp[:, 0]))

    from sklearn.metrics import roc_auc_score
    auc = roc_auc_score(cp.asnumpy(y_test_cp[:, 1]), cp.asnumpy(predictions[:, 1]))

    print(f"Test Regression MAE: {mae}")
    print(f"Test Classification AUC: {auc}")

    return mae, auc

In [22]:
from weather_prediction import WeatherPredictionNetwork
print(X_train.shape[1])
layers = [X_train.shape[1] * X_train.shape[2],512, 512, 2]
activations = ["sigmoid", "relu"]
model = WeatherPredictionNetwork(layers, activations, seed=42)

train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=5000, learning_rate=0.0001, rate = 500)

3
Epoch 0, Regression Loss: 283.9161071777344, Classification AUC: 0.7825649501275805, Learning Rate: 0.0001
Epoch 100, Regression Loss: 252.24388122558594, Classification AUC: 0.7655271398747389, Learning Rate: 0.0001
Epoch 200, Regression Loss: 212.68060302734375, Classification AUC: 0.7789491997216422, Learning Rate: 0.0001
Epoch 300, Regression Loss: 162.89266967773438, Classification AUC: 0.7900052192066807, Learning Rate: 0.0001
Epoch 400, Regression Loss: 102.13925170898438, Classification AUC: 0.7952824170726049, Learning Rate: 0.0001
Epoch 500, Regression Loss: 30.539581298828125, Classification AUC: 0.7946111691022966, Learning Rate: 1e-05
Epoch 600, Regression Loss: 22.780210494995117, Classification AUC: 0.7939196242171189, Learning Rate: 1e-05
Epoch 700, Regression Loss: 14.894908905029297, Classification AUC: 0.7945082347483182, Learning Rate: 1e-05
Epoch 800, Regression Loss: 6.892397880554199, Classification AUC: 0.7951287404314542, Learning Rate: 1e-05
Epoch 900, Regre

(array(1.6473277, dtype=float32), 0.6852876712328767)

In [23]:
# def preprocess_data(X, y):
#     X = X.reshape(X.shape[0], -1)  # Flattening the time-series data
#     X = cp.array(X, dtype=cp.float32)
#     y = cp.array(y, dtype=cp.float32)
#     return X, y


In [24]:
# from network_test import Network
# # Dataset preparation
# X_train = np.random.rand(1504, 7, 5)  # Training data (1504 samples, 7 days, 5 features per day)
# y_train = np.random.rand(1504, 2)     # Training targets (1504 samples, 2 output features)
# X_test = np.random.rand(377, 7, 5)    # Test data
# y_test = np.random.rand(377, 2)       # Test targets

# # Preprocess the data
# X_train, y_train = preprocess_data(X_train, y_train)
# X_test, y_test = preprocess_data(X_test, y_test)

# # Initialize and train the network
# activations = ["relu", "relu", "relu"] 
# nn = Network([X_train.shape[1], 64, 32, y_train.shape[1]], activations, loss_function="mse", seed=123)
# nn.train(X_train, y_train, epochs=500, learning_rate=0.001)

# # Predict and evaluate
# predictions = nn.predict(X_test)
# print("Predictions (first 5):", predictions[:5].get())
# print("Actual (first 5):", y_test[:5].get())

In [25]:
# import cupy as cp
# from network_test import Network

# # Dataset preparation
# X_train = np.random.rand(1504, 7, 5)  # Training data (1504 samples, 7 days, 5 features per day)
# y_train = np.random.rand(1504, 2)     # Training targets (1504 samples, 2 output features)
# X_test = np.random.rand(377, 7, 5)    # Test data
# y_test = np.random.rand(377, 2)       # Test targets

# # Preprocess the data (no flattening for time-series)
# def preprocess_data(X, y):
#     # Flattening the time-series data to (samples, days*features)
#     X = X.reshape(X.shape[0], -1)  # Now shape is (samples, days*features)
#     X = cp.array(X, dtype=cp.float32)
#     y = cp.array(y, dtype=cp.float32)
#     return X, y

# # Preprocess training and testing data
# X_train, y_train = preprocess_data(X_train, y_train)
# X_test, y_test = preprocess_data(X_test, y_test)

# # Initialize and train the network
# activations = ["relu", "relu", "relu"]  # Relu for hidden layers
# nn = Network([X_train.shape[1], 64, 32, y_train.shape[1]], activations, loss_function="mse", seed=123)

# # Train the network
# nn.train(X_train, y_train, epochs=500, learning_rate=0.001)

# # Predict and evaluate
# predictions = nn.predict(X_test)
# print("Predictions (first 5):", predictions[:5].get())
# print("Actual (first 5):", y_test[:5].get())


In [26]:
# import numpy as np
# import cupy as cp
# import matplotlib.pyplot as plt

# class NeuralNetwork:
#     def __init__(self, layers, activation="relu", loss="mse"):
#         self.layers = layers
#         self.activation = activation
#         self.loss = loss
#         self.weights = []
#         self.biases = []
#         self.initialize_weights()

#     def initialize_weights(self):
#         for i in range(len(self.layers) - 1):
#             weight = cp.random.randn(self.layers[i], self.layers[i + 1]) * cp.sqrt(2. / self.layers[i])
#             bias = cp.zeros((1, self.layers[i + 1]))
#             self.weights.append(weight)
#             self.biases.append(bias)

#     def activate(self, z, activation):
#         if activation == "sigmoid":
#             return 1 / (1 + cp.exp(-z))
#         elif activation == "relu":
#             return cp.maximum(0, z)
#         elif activation == "linear":
#             return z
#         else:
#             raise ValueError("Unsupported activation function")

#     def activate_derivative(self, a, activation):
#         if activation == "sigmoid":
#             return a * (1 - a)
#         elif activation == "relu":
#             return (a > 0).astype(cp.float32)
#         elif activation == "linear":
#             return cp.ones_like(a)
#         else:
#             raise ValueError("Unsupported activation function")

#     def forward(self, X):
#         activations = [X]
#         z_values = []

#         for i in range(len(self.weights)):
#             z = cp.dot(activations[-1], self.weights[i]) + self.biases[i]
#             a = self.activate(z, "linear" if i == len(self.weights) - 1 else self.activation)
#             z_values.append(z)
#             activations.append(a)

#         return activations, z_values

#     def backward(self, X, y, activations, z_values):
#         deltas = []
#         delta = activations[-1] - y
#         deltas.append(delta)

#         for i in reversed(range(len(self.weights) - 1)):
#             delta = cp.dot(deltas[0], self.weights[i + 1].T) * self.activate_derivative(activations[i + 1], self.activation)
#             deltas.insert(0, delta)

#         weight_grads = []
#         bias_grads = []

#         for i in range(len(self.weights)):
#             weight_grad = cp.dot(activations[i].T, deltas[i]) / X.shape[0]
#             bias_grad = cp.mean(deltas[i], axis=0, keepdims=True)
#             weight_grads.append(weight_grad)
#             bias_grads.append(bias_grad)

#         return weight_grads, bias_grads

#     def update_parameters(self, weight_grads, bias_grads, learning_rate):
#         for i in range(len(self.weights)):
#             self.weights[i] -= learning_rate * weight_grads[i]
#             self.biases[i] -= learning_rate * bias_grads[i]

#     def fit(self, X, y, epochs=1000, learning_rate=0.01):
#         for epoch in range(epochs):
#             activations, z_values = self.forward(X)
#             weight_grads, bias_grads = self.backward(X, y, activations, z_values)
#             self.update_parameters(weight_grads, bias_grads, learning_rate)

#             if epoch % 100 == 0:
#                 loss = self.calculate_loss(y, activations[-1])
#                 print(f"Epoch {epoch}/{epochs} - Loss: {loss}")

#     def calculate_loss(self, y, y_pred):
#         if self.loss == "mse":
#             return cp.mean((y - y_pred) ** 2)
#         else:
#             raise ValueError("Unsupported loss function")

#     def predict(self, X):
#         activations, _ = self.forward(X)
#         return activations[-1]

# def preprocess_data(X, y):
#     X = X.reshape(X.shape[0], -1)  # Flatten time-series data
#     mean_X = np.mean(X, axis=0)
#     std_X = np.std(X, axis=0)
#     X_normalized = (X - mean_X) / std_X  # Normalize inputs

#     mean_y = np.mean(y, axis=0)
#     std_y = np.std(y, axis=0)
#     y_normalized = (y - mean_y) / std_y  # Normalize targets

#     return (
#         cp.array(X_normalized, dtype=cp.float32),
#         cp.array(y_normalized, dtype=cp.float32),
#         mean_X, std_X,
#         mean_y, std_y
#     )

# def denormalize(predictions, mean_y, std_y):
#     mean_y = cp.asnumpy(mean_y)  # Convert cupy to numpy
#     std_y = cp.asnumpy(std_y)    # Convert cupy to numpy
#     predictions = np.array(predictions)  # Ensure numpy array
#     return predictions * std_y + mean_y



# # Dataset preparation
# # X_train = np.random.rand(1504, 7, 5)
# # y_train = np.random.rand(1504, 2)
# # X_test = np.random.rand(377, 7, 5)
# # y_test = np.random.rand(377, 2)

# # Preprocess the data
# X_train, y_train, mean_X, std_X, mean_y, std_y = preprocess_data(X_train, y_train)
# X_test, y_test, _, _, _, _ = preprocess_data(X_test, y_test)

# # Initialize the neural network
# nn = NeuralNetwork(
#     layers=[X_train.shape[1], 64, 32, y_train.shape[1]],  # Ensure 2 outputs for temperature and wind speed
#     activation="relu",
#     loss="mse",
# )

# # Train the model
# nn.fit(X_train, y_train, epochs=15000, learning_rate=0.01)

# # Predict normalized values
# predictions_normalized = nn.predict(X_test).get()

# # Denormalize the predictions
# predictions_denormalized = denormalize(predictions_normalized, mean_y, std_y)

# # Print results
# print("Predictions (first 5):", predictions_denormalized[:5])
# print("Actual (first 5):", y_test[:5].get())

